In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Rengarajan\\My_Projects\\DiseasePrognosis\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Rengarajan\\My_Projects\\DiseasePrognosis'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    min_samples_split: int
    min_samples_leaf: int
    random_state: int
    target_column: str

In [6]:
from diseaseprognosis.constants import *
from diseaseprognosis.utils import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            min_samples_split = params.min_samples_split,
            min_samples_leaf = params.min_samples_leaf,
            random_state = params.random_state,
            target_column = schema.name
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from diseaseprognosis import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        clf = RandomForestClassifier(n_estimators=self.config.n_estimators,
                                    min_samples_split=self.config.min_samples_split,
                                    min_samples_leaf=self.config.min_samples_leaf,
                                    random_state=42)
        clf.fit(train_x, train_y)


        joblib.dump(clf, os.path.join(self.config.root_dir, self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-12-16 19:31:21,120: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-16 19:31:21,126: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-16 19:31:21,151: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-16 19:31:21,154: INFO: common: created directory at: artifacts]
[2023-12-16 19:31:21,156: INFO: common: created directory at: artifacts/model_trainer]


InvalidParameterError: The 'max_depth' parameter of RandomForestClassifier must be an int in the range [1, inf) or None. Got 'None' instead.

In [ ]:
# this code maybe required in future
# need to make change in modeltrainer.py
train_x = train_data.drop([self.config.target_column], axis=1)
test_x = test_data.drop([self.config.target_column], axis=1)

# Use ravel() to convert to 1D array
train_y = train_data[self.config.target_column].values.ravel()
test_y = test_data[self.config.target_column].values.ravel()
